In [2]:
from dateutil.parser import *
import pymongo
import time
import pandas as pd

In [3]:
import joblib

In [4]:
data=joblib.load('old_data_0726_x2_2.pkl')

In [5]:
data1=data[['age','religion','company_city','repay_order_status']]

In [6]:
import toad

In [7]:
c = toad.transform.Combiner()

In [8]:
c.fit(data1, y = 'repay_order_status', method = 'chi', min_samples = 0.1,empty_separate = True)

In [19]:
print('age:',c.export()['age'])

age: [27, 41]


In [9]:
transer = toad.transform.WOETransformer()

In [10]:
train_woe = transer.fit_transform(c.transform(data1), data1['repay_order_status'], exclude=['repay_order_status'])

In [23]:
train_woe

,repay_order_status,age,religion,company_city
loan_id,,,,
975547,1,-0.030336,-0.066539,1.414134
975550,0,-0.360432,-0.066539,0.041888
975554,1,-0.030336,-0.066539,1.414134
975556,0,-0.030336,0.605633,-0.785220
975557,1,-0.030336,-0.066539,1.414134
...,...,...,...,...
979660,0,-0.360432,-0.066539,1.414134
979661,1,-0.360432,-0.066539,0.041888
979662,1,-0.030336,-0.066539,0.709442


In [11]:
OOT_woe = transer.transform(c.transform(data1))

In [26]:
OOT_woe

,repay_order_status,age,religion,company_city
loan_id,,,,
975547,1,-0.030336,-0.066539,1.414134
975550,0,-0.360432,-0.066539,0.041888
975554,1,-0.030336,-0.066539,1.414134
975556,0,-0.030336,0.605633,-0.785220
975557,1,-0.030336,-0.066539,1.414134
...,...,...,...,...
979660,0,-0.360432,-0.066539,1.414134
979661,1,-0.360432,-0.066539,0.041888
979662,1,-0.030336,-0.066539,0.709442


In [27]:
transer.export()['age']

{0: -0.3604321570395842, 1: -0.030335597679289547, 2: 0.5588285191549531}

In [28]:
c.export()['age']

[27, 41]

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import scale

In [13]:
df=train_woe[['age','religion','company_city','repay_order_status']]

In [22]:
x=df.iloc[:,0:-1]
y=df["repay_order_status"]
print(y.value_counts())
print("-------------------------")
print(y.value_counts(normalize=True))

1    593
0    404
Name: repay_order_status, dtype: int64
-------------------------
1    0.594784
0    0.405216
Name: repay_order_status, dtype: float64


In [15]:
def get_result_data(x,y):
    x_=scale(x,with_mean=True,with_std=True)
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=0)
    model=LogisticRegression()
    clf=model.fit(x_train,y_train)
    print("LR模型测试成绩:{:.2f}".format(clf.score(x_test,y_test)))
    y_pred=clf.predict(x_test)
    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, y_pred, target_names=target_names))
    y_pred1=clf.decision_function(x_test)
    fpr,tpr,threshold=roc_curve(y_test,y_pred1)
    rocauc=auc(fpr,tpr)#计算AUC
    print("ROC分数:{:.2f}".format(rocauc))

In [16]:
from collections import Counter
from imblearn.over_sampling import SMOTE

In [33]:
print('Original dataset shape {}'.format(Counter(y)))
oversample = SMOTE()
X_res, y_res = oversample .fit_resample(x, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({1: 593, 0: 404})
Resampled dataset shape Counter({1: 593, 0: 593})


In [18]:
get_result_data(X_res, y_res)

LR模型测试成绩:0.67
              precision    recall  f1-score   support

     class 0       0.76      0.50      0.60       238
     class 1       0.62      0.84      0.72       237

    accuracy                           0.67       475
   macro avg       0.69      0.67      0.66       475
weighted avg       0.69      0.67      0.66       475

ROC分数:0.73


In [28]:
x_=scale(x,with_mean=True,with_std=True)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=0)

In [29]:
model=LogisticRegression(class_weight="balanced")
clf=model.fit(x_train,y_train)
print("LR模型测试成绩:{:.2f}".format(clf.score(x_test,y_test)))
y_pred=clf.predict(x_test)

LR模型测试成绩:0.70


In [30]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

y_pred1=clf.decision_function(x_test)
fpr,tpr,threshold=roc_curve(y_test,y_pred1)
rocauc=auc(fpr,tpr)#计算AUC
print("ROC分数:{:.2f}".format(rocauc))

              precision    recall  f1-score   support

     class 0       0.62      0.71      0.66       167
     class 1       0.77      0.69      0.73       232

    accuracy                           0.70       399
   macro avg       0.70      0.70      0.70       399
weighted avg       0.71      0.70      0.70       399

ROC分数:0.77
